In [139]:
import pandas as pd
import numpy as np

In [140]:
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline

In [141]:
from sklearn.model_selection import train_test_split

In [142]:
# Import the classifier from sklearn
from sklearn.tree import DecisionTreeClassifier

In [143]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [144]:
from sklearn.metrics import precision_recall_fscore_support

In [145]:
og = pd.read_csv(r"C:\Users\Matt\Desktop\eur_data.csv")

In [146]:
len(og)

1953

In [147]:
def vol_convert(vol):
    rate = vol/1600
    return rate

In [148]:
start_vol = 4
end_vol = 10
step = .15
hist_period1 = 10
hist_period2 = 20
hist_period3 = 30
volume_period1 = 2
volume_period2 = 4
forward_vol_period = 20
vol_to_test = 6
rate = vol_convert(vol_to_test)

In [149]:
og.tail()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest
1948,9/24/2019,1.10585,1.10925,1.10490,1.10855,0.00230,156293,502612
1949,9/25/2019,1.10855,1.10880,1.10035,1.10075,-0.00780,177191,506913
1950,9/26/2019,1.10090,1.10315,1.09720,1.09920,-0.00155,192688,503940
1951,9/27/2019,1.09800,1.10200,1.09660,1.10010,0.00090,163496,504950
1952,9/30/2019,1.09995,1.10080,1.09445,1.09500,-0.00510,165000,504950


In [150]:
data = og.copy()

In [151]:
#process dataset ready for machine learning, add custome columns, forward vol uses mean close to close and abs change 

def process(dataframe, hist1, hist2, hist3, volume1, volume2, forward_vol_period, question_vol, cushion):
    
    
    dataframe['abs_change'] = dataframe['Change'].abs() / dataframe['Close']
    
    dataframe['high_move'] = (((dataframe.High - dataframe.Close.shift(1))/dataframe['Close']).abs())
    dataframe['low_move'] = (((dataframe.Low - data.Close.shift(1))/dataframe['Close']).abs())
    dataframe['max_move'] = dataframe[['high_move', 'low_move']].max(axis=1)
    
    dataframe['hist_max_1'] = dataframe.max_move.rolling(window=hist1).mean()
    dataframe['hist_max_2'] = dataframe.max_move.rolling(window=hist2).mean()
    dataframe['hist_max_3'] = dataframe.max_move.rolling(window=hist3).mean()
    
    dataframe['hist_change_1'] = dataframe.abs_change.rolling(window=hist1).mean()
    dataframe['hist_change_2'] = dataframe.abs_change.rolling(window=hist2).mean()
    dataframe['hist_change_3'] = dataframe.abs_change.rolling(window=hist3).mean()
    
    dataframe['avg_max_close'] = dataframe[['max_move', 'abs_change']].mean(axis=1)
    
    dataframe['hist_avgmax_1'] = dataframe.avg_max_close.rolling(window=hist1).mean()
    dataframe['hist_avgmax_2'] = dataframe.avg_max_close.rolling(window=hist2).mean()
    dataframe['hist_avgmax_3'] = dataframe.avg_max_close.rolling(window=hist3).mean()
    
    dataframe['hist_volume_1'] = dataframe.Volume.rolling(window=volume1).mean()
    dataframe['hist_volume_2'] = dataframe.Volume.rolling(window=volume2).mean()
    
    dataframe['backtothefuture'] = dataframe.avg_max_close.rolling(window=forward_vol_period).mean()
    
    dataframe['forward_avg_close_max'] = dataframe['backtothefuture'].shift(-forward_vol_period)
    
    
    
    
    
    
    dataframe['volatile'] = (dataframe['forward_avg_close_max'] > question_vol + cushion)
    dataframe = dataframe.applymap(lambda x: 1 if x == True else x)
    dataframe = dataframe.applymap(lambda x: 0 if x == False else x)
    #drop column 
    del dataframe['Date Time']
    del dataframe['Open']
    del dataframe['High']
    del dataframe['Low']
    del dataframe['Close']
    del dataframe['backtothefuture']
    
    return dataframe 

In [152]:
#set and process dataset 
new_data = process(data,hist_period1 ,hist_period2 ,hist_period3 ,volume_period1,volume_period2,forward_vol_period, rate, 0)

In [153]:
new_data.tail()

,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,...,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2,forward_avg_close_max,volatile
1948,0.00230,156293,502612,0.002075,0.002706,0.001218,0.002706,0.005633,0.004732,0.004355,...,0.003075,0.002895,0.002391,0.004822,0.003903,0.003625,162367.0,158377.75,NaN,0
1949,-0.00780,177191,506913,0.007086,0.000227,0.007449,0.007449,0.005811,0.005028,0.004449,...,0.003402,0.002991,0.007268,0.005081,0.004215,0.003720,166742.0,164154.00,NaN,0
1950,-0.00155,192688,503940,0.001410,0.002183,0.003230,0.003230,0.004779,0.005094,0.004421,...,0.003408,0.002928,0.002320,0.004019,0.004251,0.003674,184939.5,173653.25,NaN,0
1951,0.00090,163496,504950,0.000818,0.002545,0.002363,0.002545,0.004720,0.005036,0.004353,...,0.003311,0.002852,0.001682,0.003997,0.004173,0.003602,178092.0,172417.00,NaN,0
1952,-0.00510,165000,504950,0.004658,0.000639,0.005160,0.005160,0.004568,0.004887,0.004398,...,0.003200,0.002975,0.004909,0.003881,0.004044,0.003686,164248.0,174593.75,NaN,0


In [154]:
new_data['forward_avg_close_max'].describe()

count    1933.000000
mean        0.005002
std         0.001509
min         0.001984
25%         0.003999
50%         0.004784
75%         0.005786
max         0.010881
Name: forward_avg_close_max, dtype: float64

In [155]:
new_data['volatile'].value_counts()

1    1585
0     368
Name: volatile, dtype: int64

In [156]:
new_data = new_data.dropna(how = 'any')

In [157]:
new_data.head(20)

,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,...,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2,forward_avg_close_max,volatile
30,-0.0031,321995,299085,0.002373,0.007502,0.003751,0.007502,0.007685,0.007899,0.008707,...,0.004179,0.005276,0.004938,0.005789,0.006039,0.006991,302346.0,265401.50,0.006534,1
31,0.0078,359743,294462,0.005936,0.007458,0.006697,0.007458,0.007882,0.007750,0.008546,...,0.004101,0.005168,0.006697,0.006128,0.005926,0.006857,340869.0,293553.00,0.006541,1
32,0.0018,233219,295500,0.001368,0.004408,0.001900,0.004408,0.007745,0.007769,0.008257,...,0.004123,0.004817,0.002888,0.006086,0.005946,0.006537,296481.0,299413.50,0.006696,1
33,0.0088,385308,287525,0.006643,0.010266,0.000000,0.010266,0.007812,0.007779,0.008367,...,0.004090,0.004874,0.008455,0.006341,0.005935,0.006620,309263.5,325066.25,0.006417,1
34,-0.0002,238653,287495,0.000151,0.001586,0.002643,0.002643,0.006906,0.007666,0.008293,...,0.004079,0.004795,0.001397,0.005432,0.005872,0.006544,311980.5,304230.75,0.006502,1
35,0.0091,322144,280074,0.006824,0.010198,0.000975,0.010198,0.007609,0.007798,0.008468,...,0.004183,0.004934,0.008511,0.006105,0.005990,0.006701,280398.5,294831.00,0.006251,1
36,0.0124,265826,276830,0.009212,0.011293,0.001634,0.011293,0.008197,0.007985,0.008501,...,0.004556,0.004992,0.010253,0.006720,0.006270,0.006746,293985.0,302982.75,0.006104,1
37,-0.0062,261685,272859,0.004628,0.000970,0.006941,0.006941,0.007881,0.007851,0.008348,...,0.004420,0.004803,0.005784,0.006346,0.006135,0.006576,263755.5,272077.00,0.006158,1
38,0.0060,253899,271282,0.004458,0.005573,0.000594,0.005573,0.007583,0.007638,0.007995,...,0.004334,0.004531,0.005016,0.006295,0.005986,0.006263,257792.0,275888.50,0.006001,1
39,-0.0119,351274,275227,0.008921,0.002174,0.010720,0.010720,0.007700,0.007830,0.007967,...,0.004628,0.004692,0.009821,0.006376,0.006229,0.006329,302586.5,283171.00,0.005645,1


In [158]:
# Store the 'Survived' feature in a new variable and remove it from the dataset
outcomes = new_data['volatile']


del new_data['volatile']
del new_data['forward_avg_close_max']


features = new_data

# Show the new dataset with 'Survived' removed
features.head()

,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,hist_change_1,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2
30,-0.0031,321995,299085,0.002373,0.007502,0.003751,0.007502,0.007685,0.007899,0.008707,0.003894,0.004179,0.005276,0.004938,0.005789,0.006039,0.006991,302346.0,265401.50
31,0.0078,359743,294462,0.005936,0.007458,0.006697,0.007458,0.007882,0.007750,0.008546,0.004373,0.004101,0.005168,0.006697,0.006128,0.005926,0.006857,340869.0,293553.00
32,0.0018,233219,295500,0.001368,0.004408,0.001900,0.004408,0.007745,0.007769,0.008257,0.004427,0.004123,0.004817,0.002888,0.006086,0.005946,0.006537,296481.0,299413.50
33,0.0088,385308,287525,0.006643,0.010266,0.000000,0.010266,0.007812,0.007779,0.008367,0.004870,0.004090,0.004874,0.008455,0.006341,0.005935,0.006620,309263.5,325066.25
34,-0.0002,238653,287495,0.000151,0.001586,0.002643,0.002643,0.006906,0.007666,0.008293,0.003957,0.004079,0.004795,0.001397,0.005432,0.005872,0.006544,311980.5,304230.75


In [159]:
X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=0.30, random_state=42)

In [160]:
# TODO: Define the classifier, and fit it to the data
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [161]:
# Making predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate the accuracy
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('The training accuracy is', train_accuracy)
print('The test accuracy is', test_accuracy)

The training accuracy is 1.0
The test accuracy is 0.9106830122591943


In [162]:
# Training the model
model = DecisionTreeClassifier(max_depth=15, min_samples_leaf=20, min_samples_split=20)
model.fit(X_train, y_train)

# Making predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculating accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('The training accuracy is', train_accuracy)
print('The test accuracy is', test_accuracy)

The training accuracy is 0.9279279279279279
The test accuracy is 0.8914185639229422


In [163]:
def process_sample(dataframe, hist1, hist2, hist3, volume1, volume2):
    
    dataframe['abs_change'] = dataframe['Change'].abs() / dataframe['Close']
    
    dataframe['high_move'] = (((dataframe.High - dataframe.Close.shift(1))/dataframe['Close']).abs())
    dataframe['low_move'] = (((dataframe.Low - data.Close.shift(1))/dataframe['Close']).abs())
    dataframe['max_move'] = dataframe[['high_move', 'low_move']].max(axis=1)
    
    dataframe['hist_max_1'] = dataframe.max_move.rolling(window=hist1).mean()
    dataframe['hist_max_2'] = dataframe.max_move.rolling(window=hist2).mean()
    dataframe['hist_max_3'] = dataframe.max_move.rolling(window=hist3).mean()
    
    dataframe['hist_change_1'] = dataframe.abs_change.rolling(window=hist1).mean()
    dataframe['hist_change_2'] = dataframe.abs_change.rolling(window=hist2).mean()
    dataframe['hist_change_3'] = dataframe.abs_change.rolling(window=hist3).mean()
    
    dataframe['avg_max_close'] = dataframe[['max_move', 'abs_change']].mean(axis=1)
    
    dataframe['hist_avgmax_1'] = dataframe.avg_max_close.rolling(window=hist1).mean()
    dataframe['hist_avgmax_2'] = dataframe.avg_max_close.rolling(window=hist2).mean()
    dataframe['hist_avgmax_3'] = dataframe.avg_max_close.rolling(window=hist3).mean()
    
    dataframe['hist_volume_1'] = dataframe.Volume.rolling(window=volume1).mean()
    dataframe['hist_volume_2'] = dataframe.Volume.rolling(window=volume2).mean()
    

    #drop column 
    del dataframe['Open']
    del dataframe['High']
    del dataframe['Low']
    del dataframe['Close']
    
    return dataframe 

In [164]:
new_og = process_sample(og, hist_period1 ,hist_period2 ,hist_period3 , volume_period1, volume_period2)

In [165]:
new_og.tail()

,Date Time,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,hist_change_1,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2
1948,9/24/2019,0.00230,156293,502612,0.002075,0.002706,0.001218,0.002706,0.005633,0.004732,0.004355,0.004011,0.003075,0.002895,0.002391,0.004822,0.003903,0.003625,162367.0,158377.75
1949,9/25/2019,-0.00780,177191,506913,0.007086,0.000227,0.007449,0.007449,0.005811,0.005028,0.004449,0.004352,0.003402,0.002991,0.007268,0.005081,0.004215,0.003720,166742.0,164154.00
1950,9/26/2019,-0.00155,192688,503940,0.001410,0.002183,0.003230,0.003230,0.004779,0.005094,0.004421,0.003259,0.003408,0.002928,0.002320,0.004019,0.004251,0.003674,184939.5,173653.25
1951,9/27/2019,0.00090,163496,504950,0.000818,0.002545,0.002363,0.002545,0.004720,0.005036,0.004353,0.003274,0.003311,0.002852,0.001682,0.003997,0.004173,0.003602,178092.0,172417.00
1952,9/30/2019,-0.00510,165000,504950,0.004658,0.000639,0.005160,0.005160,0.004568,0.004887,0.004398,0.003194,0.003200,0.002975,0.004909,0.003881,0.004044,0.003686,164248.0,174593.75


In [166]:
del new_og['Date Time']

In [167]:
new_og.tail()

,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,hist_change_1,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2
1948,0.00230,156293,502612,0.002075,0.002706,0.001218,0.002706,0.005633,0.004732,0.004355,0.004011,0.003075,0.002895,0.002391,0.004822,0.003903,0.003625,162367.0,158377.75
1949,-0.00780,177191,506913,0.007086,0.000227,0.007449,0.007449,0.005811,0.005028,0.004449,0.004352,0.003402,0.002991,0.007268,0.005081,0.004215,0.003720,166742.0,164154.00
1950,-0.00155,192688,503940,0.001410,0.002183,0.003230,0.003230,0.004779,0.005094,0.004421,0.003259,0.003408,0.002928,0.002320,0.004019,0.004251,0.003674,184939.5,173653.25
1951,0.00090,163496,504950,0.000818,0.002545,0.002363,0.002545,0.004720,0.005036,0.004353,0.003274,0.003311,0.002852,0.001682,0.003997,0.004173,0.003602,178092.0,172417.00
1952,-0.00510,165000,504950,0.004658,0.000639,0.005160,0.005160,0.004568,0.004887,0.004398,0.003194,0.003200,0.002975,0.004909,0.003881,0.004044,0.003686,164248.0,174593.75


In [168]:
last_date = new_og.index[-1]
last_date

1952

In [169]:
one_sample = [new_og.iloc[last_date,:]] 

In [170]:
model.predict(one_sample)

array([0], dtype=int64)

In [171]:
def tail_vol(period, dataframe):
    rate = dataframe['avg_max_close'].tail(period).mean()
    vol = round((rate*100*16),2)
    
    return vol

In [172]:
tail_vol(20, new_og)

6.47

In [173]:
answer = model.predict(one_sample)
answer = answer[0]
answer

0

In [174]:
def find_vol(dataframe, start_vol, end_vol, step, hist_period1, hist_period2, hist_period3, 
             volume_period1, volume_period2, forward_vol_period):
    vol = start_vol
    while vol < end_vol: 
        answer = 1 
        rate = vol_convert(vol)
        beg_frame = dataframe.copy()
        
        frame = process(dataframe,hist_period1 ,hist_period2 ,hist_period3 ,volume_period1,volume_period2,
                        forward_vol_period, rate, 0)
   
    
        frame = frame.dropna(how = 'any')
    
        outcomes = frame['volatile']
        features = frame

        del frame['volatile']
        del frame['forward_avg_close_max']
        X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=0.30, random_state=42)
        # Training the model
        model = DecisionTreeClassifier(max_depth=15, min_samples_leaf=20, min_samples_split=20)
        model.fit(X_train, y_train)

        # Making predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Calculating accuracies
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        recall = recall_score(y_test, y_test_pred)
        the_f1 = f1_score(y_test, y_test_pred)
        
   
    
        original_data = process_sample(beg_frame, hist_period1 ,hist_period2 ,hist_period3 , volume_period1, volume_period2)
        #original_data = original_data.dropna(how = 'any')
        
        del original_data['Date Time']
        last_row = original_data.index[-1]
        one_sample = [new_og.iloc[last_row,:]] 
    
        answer = model.predict(one_sample)
        answer = answer[0]
        print(vol)
        print(answer)
        print('The training accuracy is', train_accuracy)
        print('The test accuracy is', test_accuracy)
        print('The precision is', precision)
        print('The recall is', recall)
        print('The F1 is', the_f1 )
        
        
        print('The number of days', len(outcomes))
        print('The percentage of volatile days', ((outcomes == 1).sum()) / len(outcomes))
        
        
        
        #print(frame.head(1))
    
        #if answer == 0:
            #break
        
        vol = vol + step


In [175]:
fv_dataframe = pd.read_csv(r"C:\Users\Matt\Desktop\eur_data.csv")
vol = 5.75
start_vol = 4.50
end_vol = 7
step = .25
hist_period1 = 10
hist_period2 = 15
hist_period3 = 20
volume_period1 = 4
volume_period2 = 8
forward_vol_period = 25
rate = vol_convert(vol)

In [176]:
find_vol(fv_dataframe, start_vol, end_vol, step, hist_period1, hist_period2, hist_period3, 
         volume_period1, volume_period2, forward_vol_period)

4.5
1
The training accuracy is 0.9775280898876404
The test accuracy is 0.9685863874345549
The precision is 0.9796296296296296
The recall is 0.9869402985074627
The F1 is 0.983271375464684
The number of days 1908
The percentage of volatile days 0.9481132075471698
4.75
1
The training accuracy is 0.9715355805243445
The test accuracy is 0.9546247818499127
The precision is 0.9845559845559846
The recall is 0.9659090909090909
The F1 is 0.9751434034416827
The number of days 1908
The percentage of volatile days 0.9350104821802935
5.0
1
The training accuracy is 0.9655430711610486
The test accuracy is 0.9458987783595113
The precision is 0.9602272727272727
The recall is 0.9806576402321083
The F1 is 0.970334928229665
The number of days 1908
The percentage of volatile days 0.9229559748427673
5.25
1
The training accuracy is 0.9610486891385768
The test accuracy is 0.9493891797556719
The precision is 0.9634615384615385
The recall is 0.9804305283757339
The F1 is 0.9718719689621726
The number of days 1908

In [177]:
ex = process(fv_dataframe,hist_period1 ,hist_period2 ,hist_period3 ,volume_period1,volume_period2,
                        forward_vol_period, rate, 0)

In [178]:
del ex['forward_avg_close_max']
del ex['volatile']

test_rate = vol_convert(vol)

ex['volatile'] = (ex['avg_max_close'] > test_rate)

ex = ex.applymap(lambda x: 0 if x == False else x)
ex = ex.applymap(lambda x: 1 if x == True else x)

In [179]:
ex = ex.dropna(how = 'any')
len(ex)

1933

In [180]:
print('the 5 day scalp average vol is', tail_vol(5, ex) )
print('the 10 day scalp average vol is', tail_vol(10, ex) )
print('the 20 day scalp average vol is', tail_vol(20, ex) )
print('the 40 day scalp average vol is', tail_vol(40, ex) )
print('the 60 day scalp average vol is', tail_vol(60, ex) )
print('the 90 day scalp average vol is', tail_vol(40, ex) )
print('the 120 day scalp average vol is', tail_vol(60, ex) )


the 5 day scalp average vol is 5.94
the 10 day scalp average vol is 6.21
the 20 day scalp average vol is 6.47
the 40 day scalp average vol is 5.85
the 60 day scalp average vol is 5.42
the 90 day scalp average vol is 5.85
the 120 day scalp average vol is 5.42


In [181]:
fourty = ex.tail(40)
thirty = ex.tail(30)
twenty = ex.tail(20)
ten = ex.tail(10)
five = ex.tail(5)

#new['volatile'].sum()/len(new)

In [182]:
print('vol is', vol)
print('the percentage of volatile days over the last 5 days:', five['volatile'].sum()/len(five) )
print('the percentage of volatile days over the last 10 days:', ten['volatile'].sum()/len(ten) )
print('the percentage of volatile days over the last 20 days:', twenty['volatile'].sum()/len(twenty) )
print('the percentage of volatile days over the last 30 days:', thirty['volatile'].sum()/len(thirty) )
print('the percentage of volatile days over the last 40 days:', fourty['volatile'].sum()/len(fourty) )


vol is 5.75
the percentage of volatile days over the last 5 days: 0.4
the percentage of volatile days over the last 10 days: 0.5
the percentage of volatile days over the last 20 days: 0.45
the percentage of volatile days over the last 30 days: 0.4
the percentage of volatile days over the last 40 days: 0.4


In [183]:
five.tail(1)

,Change,Volume,Open Interest,abs_change,high_move,low_move,max_move,hist_max_1,hist_max_2,hist_max_3,hist_change_1,hist_change_2,hist_change_3,avg_max_close,hist_avgmax_1,hist_avgmax_2,hist_avgmax_3,hist_volume_1,hist_volume_2,volatile
1952,-0.0051,165000,504950,0.004658,0.000639,0.00516,0.00516,0.004568,0.005117,0.004887,0.003194,0.003641,0.0032,0.004909,0.003881,0.004379,0.004044,174593.75,166485.75,1


In [184]:
len(ex)

1933